# DDIM -- diffusers
前面的章节中已经介绍了 DDPM 的数学原理以及代码实现，接下来将带来 DDIM 相关的代码实验。

In [ ]:
import torch
from diffusers import DDPMScheduler, DDIMScheduler, UNet2DModel

device = "cuda"
model_id = "/mnt/d/Model_Data/pretrain_model/ddpm-cat"

model = UNet2DModel.from_pretrained(model_id).to(device).eval()
ddpm = DDPMScheduler.from_pretrained(model_id)
ddim = DDIMScheduler.from_config(ddpm.config)

使用 DDPM 的可视化工具

In [ ]:
def add_text_to_image(image_path, output_path, title, save=False):
    from PIL import Image, ImageDraw, ImageFont
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype("DejaVuSans-Bold.ttf", 36)
    text_position = (10,10)
    draw.text(text_position, title, font=font, fill='white')
    if save: 
        image.save(output_path)
    return image

def savefig(input, name):
    from PIL import Image
    image = (input / 2 + 0.5).clamp(0, 1).squeeze()
    if len(image.shape)==3:
        image = (image.permute(1, 2, 0) * 255).round().to(torch.uint8).cpu().numpy()
    else:
        image = (image * 255).round().to(torch.uint8).cpu().numpy()
    image = Image.fromarray(image)
    image.save(name)

In [ ]:
from tqdm import tqdm

@torch.inference_mode()
def sample_with(scheduler, num_inference_steps=50, eta=0.0, seed=0, batch_size=1):
    
    scheduler.set_timesteps(num_inference_steps, device=device)
    g = torch.Generator(device=device).manual_seed(seed)
    sample_size = model.config.sample_size
    channels = model.config.in_channels
    
    x = torch.randn(batch_size, channels, sample_size, sample_size, generator=g, device=device)

    for t in tqdm(scheduler.timesteps, desc=f"DDPM-step:{num_inference_steps}" if isinstance(scheduler, DDPMScheduler) else f"DDIM-step:{num_inference_steps}-eta:{eta}"):
        # 某些调度器需要对输入做缩放（DDPM/DDIM 多数情况下为恒等，但兼容写法更稳妥）
        model_input = scheduler.scale_model_input(x, t)
        noise_pred = model(model_input, t).sample

        if isinstance(scheduler, DDIMScheduler):
            x = scheduler.step(noise_pred, t, x, eta=eta).prev_sample
        else:
            x = scheduler.step(noise_pred, t, x).prev_sample

    return x

# ====== 对比使用 ======
# 1) DDPM 基线（例如 1000 步）
imgs_ddpm_1000 = sample_with(ddpm, num_inference_steps=1000, seed=42)
# savefig(imgs_ddpm_1000, "DDPM-1000.png")

imgs_ddpm_100 = sample_with(ddpm, num_inference_steps=100, seed=42)
# savefig(imgs_ddpm_100, "DDPM-100.png")

# 2) DDIM（比如 50 或 100 步；eta=0 为确定性）
imgs_ddim_50 = sample_with(ddim, num_inference_steps=50, eta=0.0, seed=42)
# savefig(imgs_ddim_50, "DDIM-50.png")

imgs_ddim_100 = sample_with(ddim, num_inference_steps=100, eta=0.0, seed=42)
# savefig(imgs_ddim_100, "DDIM-100.png")

# 如果想看带随机性的 DDIM，可以把 eta 调到 0.1~0.2 试试
imgs_ddim_stochastic = sample_with(ddim, num_inference_steps=50, eta=1, seed=42)
# savefig(imgs_ddim_stochastic, "DDIM-stochastic.png")


In [ ]:
savefig(imgs_ddpm_1000, "DDPM-1000.png")
savefig(imgs_ddpm_100, "DDPM-100.png")
savefig(imgs_ddim_50, "DDIM-50.png")
savefig(imgs_ddim_100, "DDIM-100.png")
savefig(imgs_ddim_stochastic, "DDIM-stochastic.png")